In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from queue import Queue
from queue import PriorityQueue

def draw_search_tree(adj_list, search_method, start, goal=None, heuristic=None):
    """
    Draws the search tree generated by the specified search method.

    Parameters:
        adj_list (dict): adjacency list {node: [(neighbor, cost), ...]}
        search_method (str): 'DFS', 'BFS', or 'A*'
        start: starting node
        goal: goal node (optional, required for A*)
        heuristic (function): heuristic function h(node) for A* (optional)
    """
    tree = nx.DiGraph()
    visited = set()
    parent = {}

    if search_method.upper() == "DFS":
        stack = [start]
        parent[start] = None
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                for neighbor, _ in adj_list.get(node, []):
                    if neighbor not in visited:
                        stack.append(neighbor)
                        parent[neighbor] = node

    elif search_method.upper() == "BFS":
        q = Queue()
        q.put(start)
        parent[start] = None
        visited.add(start)
        while not q.empty():
            node = q.get()
            for neighbor, _ in adj_list.get(node, []):
                if neighbor not in visited:
                    visited.add(neighbor)
                    q.put(neighbor)
                    parent[neighbor] = node

    elif search_method.upper() == "A*":
        if heuristic is None or goal is None:
            raise ValueError("A* search requires a heuristic function and a goal node.")
        pq = PriorityQueue()
        pq.put((heuristic(start), 0, start))
        parent[start] = None
        g_score = {start: 0}
        while not pq.empty():
            _, cost, node = pq.get()
            if node in visited:
                continue
            visited.add(node)
            if node == goal:
                break
            for neighbor, edge_cost in adj_list.get(node, []):
                tentative_g = g_score[node] + edge_cost
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + heuristic(neighbor)
                    pq.put((f_score, tentative_g, neighbor))
                    parent[neighbor] = node

    else:
        raise ValueError("search_method must be 'DFS', 'BFS', or 'A*'")

    # Build the search tree
    for child, par in parent.items():
        if par is not None:
            tree.add_edge(par, child)
        else:
            tree.add_node(child)

    # Use hierarchical layout for tree-like appearance
    plt.figure(figsize=(10, 8))

    # Create a layout without using pygraphviz
    # Option 1: Use spring layout with parameters tuned for tree-like appearance
    pos = nx.spring_layout(tree, k=0.8, iterations=100, seed=42)
    
    # Alternatively, you can use other layouts like:
    # pos = nx.kamada_kawai_layout(tree)
    # pos = nx.shell_layout(tree)

    # Draw the graph with the layout
    nx.draw(
        tree,
        pos,
        with_labels=True,
        arrows=True,
        node_color="lightblue",
        node_size=800,
        font_size=10,
        font_weight="bold",
        arrowsize=15,
    )

    plt.title(f"{search_method.upper()} Search Tree", fontsize=16)
    plt.axis("off")  # Turn off axis
    plt.tight_layout()
    plt.show()

In [7]:
search_graph = {
    'A': [('B', 2), ('C', 2), ('G', 7)],
    'B': [('A', 2), ('D', 2)],
    'C': [('A', 2), ('E', 2)],
    'D': [('B', 2), ('F', 2)],
    'E': [('C', 2), ('H', 2)],
    'F': [('D', 2), ('I', 2)],
    'G': [('A', 7), ('J', 2)],
    'H': [('E', 2), ('J', 6)],
    'I': [('F', 2), ('J', 8)],
    'J': [('G', 2), ('H', 6), ('I', 8)],
}

heuristic = lambda x: {
    'A': 6,
    'B': 6,
    'C': 5,
    'D': 5,
    'E': 3,
    'F': 4,
    'G': 2,
    'H': 3,
    'I': 4,
    'J': 0
}[x]

draw_search_tree(search_graph, 'DFS', 'A', 'J', heuristic)

ImportError: requires pygraphviz http://pygraphviz.github.io/

<Figure size 1000x800 with 0 Axes>